In [1]:
from sqlalchemy import create_engine
import os
import pandas as pd

In [2]:
os.getcwd()

'E:\\SpringBoard\\Study Material\\DSC\\Case Studies\\Country_Club'

In [3]:
engine = create_engine('sqlite:///country_club.sqlite')
con = engine.connect()

In [4]:
engine.table_names()

['Bookings', 'Facilities', 'Members']

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [5]:
query1 = 'select * from Facilities where membercost>0'
df1 = pd.read_sql(query1,con)
df1.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,4,Massage Room 1,9.9,80.0,4000,3000
3,5,Massage Room 2,9.9,80.0,4000,3000
4,6,Squash Court,3.5,17.5,5000,80


Q2: How many facilities do not charge a fee to members?

In [6]:
query2 = 'select count(facid) from Facilities where membercost=0'
df2 = pd.read_sql(query2,con)
df2

,count(facid)
0,4


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [7]:
query3 = r'select facid, name, membercost, monthlymaintenance from Facilities where membercost<= monthlymaintenance*.2'
df3 = pd.read_sql(query3, con)
df3.head()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [8]:
query4 = r'select * from Facilities where facid in (1,5)'
df4 = pd.read_sql(query4, con)
df4.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [9]:
q5 = r'''select name, monthlymaintenance, case when monthlymaintenance>=100 then 'expensive' else 'cheap' end as Facility_class
        from Facilities'''
df5 = pd.read_sql(q5,con)
df5.head()

,name,monthlymaintenance,Facility_class
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [10]:
q6 = r"""select firstname, surname from Members"""
df6 = pd.read_sql(q6,con)
df6.head()

,firstname,surname
0,GUEST,GUEST
1,Darren,Smith
2,Tracy,Smith
3,Tim,Rownam
4,Janice,Joplette


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [11]:
q7 = r"""select distinct f.name, (firstname||' '||surname) as Member from Members m
        inner join Bookings b on m.memid = b.memid
        inner join Facilities f on b.facid = f.facid

        where f.name like ('Tennis%')

        order by Member
        """

df7 = pd.read_sql(q7,con)
df7.head()

,name,Member
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen


Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [12]:
q8 = r"""select f.name, (firstname||' '||surname) as member,
        case when b.memid = 0 then f.guestcost*b.slots
        else f.membercost*b.slots end as cost

        from Bookings as b
        inner join Facilities f on b.facid = f.facid
        inner join Members m on b.memid = m.memid

        where date(starttime) = '2012-09-14' and cost >=30
        order by cost desc"""

df8 = pd.read_sql(q8, con)
df8.head(20)

,name,member,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Q9: This time, produce the same result as in Q8, but using a subquery. 
** Skipping Q9 **

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [13]:
q10 = """select f.name, sum(case when b.memid = 0 then f.guestcost*b.slots
        else f.membercost*b.slots end) as total_revenue

        from Bookings as b
        inner join Facilities f on b.facid = f.facid
        inner join Members m on b.memid = m.memid

group by f.name 
having total_revenue <=1000"""

df10 = pd.read_sql(q10, con)
df10.head()

,name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [14]:
q11 = r"""
        select m.surname, m.firstname, (rm.surname||' '|| rm.firstname) as recommendedby
        from Members m inner join Members rm on m.recommendedby = rm.memid

        where m.memid <> 0
        order by m.surname, m.firstname"""

df11 = pd.read_sql(q11, con)
df11.head()

,surname,firstname,recommendedby
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren


Q12: Find the facilities with their usage by member, but not guests

In [15]:
q12 = r"""select f.name, count(b.facid) as total_usage
        from Bookings b
        inner join Facilities f on b.facid = f.facid

        where b.memid <> 0
        group by f.name
        order by total_usage desc"""

df12 = pd.read_sql(q12,con)
df12.head()

,name,total_usage
0,Pool Table,783
1,Massage Room 1,421
2,Snooker Table,421
3,Table Tennis,385
4,Badminton Court,344


Q13: Find the facilities usage by month, but not guests

In [16]:
q13 = r"""select f.name,strftime('%m',starttime)||'-'||strftime('%Y',starttime) as month_yr, count(b.facid) as total_usage
        from Bookings b
        inner join Facilities f on b.facid = f.facid

        where b.memid <> 0
        group by f.name, month_yr
        order by total_usage desc"""

df13 = pd.read_sql(q13, con)
df13.head()

,name,month_yr,total_usage
0,Pool Table,09-2012,408
1,Pool Table,08-2012,272
2,Snooker Table,09-2012,199
3,Table Tennis,09-2012,194
4,Massage Room 1,09-2012,191
